In [1]:
from datetime import datetime, timezone
import ape
import pickle
from ape import chain, networks, Contract
import pandas as pd

In [2]:
def flatten(logs):
    flat = []
    for log in logs:
        t = log.event_arguments
        t["block_number"] = log.block_number
        t["transaction_hash"] = log.transaction_hash.hex()
        flat.append(t)
    return pd.DataFrame(flat)

networks.parse_network_choice("ethereum:mainnet:geth").__enter__()

INFO: Connecting to existing Geth node at 'http://fullnode.dappnode:8545'.


<geth chain_id=1>

In [3]:
# Do a binary search to find the timing. 
def findBlockNumberFromTime(target = datetime(2022, 1, 1, 12, 0, 0, 0, timezone.utc)):
    lower = 0
    higher = curr = chain.blocks.height

    while lower + 1 != higher:
        curr_block = lower + (higher - lower) // 2
        curr = datetime.fromtimestamp(
            networks.active_provider.get_block(curr_block).timestamp, timezone.utc
        )

        if curr < target:
            lower = curr_block
        else:
            higher = curr_block

    if curr < target:
        curr_block += 1

    curr = datetime.fromtimestamp(
        networks.active_provider.get_block(curr_block).timestamp, timezone.utc
    )
    
    return (curr_block, curr)

In [4]:
(start_block_number, start_time) = findBlockNumberFromTime(datetime(2022, 7, 22, 12, 0, 0, 0, timezone.utc))
(end_block_number, end_time) = findBlockNumberFromTime(datetime(2022, 7, 29, 12, 0, 0, 0, timezone.utc))

cutoff_eth = 1e18
cutoff_dai = 2000e18

In [5]:
aztec_connect = Contract("rollup.aztec.eth")
connect_deployed = 14923081

if True:
    ac_deposits = flatten(aztec_connect.Deposit.range(
        start_or_stop=max(connect_deployed, start_block_number), 
        stop = end_block_number, 
        block_page_size = 10000
    ))
    pickle.dump(ac_deposits, open("ac_deposits.p", "wb"))
ac_deposits = pickle.load(open("ac_deposits.p", "rb"))

In [10]:
cutoff_block_start = start_block_number
cutoff_block_end = end_block_number

in_time = ac_deposits[
    (ac_deposits["block_number"] >= cutoff_block_start) & 
    (ac_deposits["block_number"] <= cutoff_block_end)
]
eligible = in_time.drop(["block_number", "transaction_hash"], axis=1)
eligible = eligible.groupby(["assetId", "depositorAddress"]).sum()
eligible = eligible.reset_index(level=[0, 1])
eligible = eligible[
    ((eligible["assetId"] == 0) & (eligible["depositValue"] >= cutoff_eth)) |
    ((eligible["assetId"] == 1) & (eligible["depositValue"] >= cutoff_dai))
]

winner = eligible.sample(random_state=cutoff_block_end).values[0]

In [11]:
print("For the timeframe {0} (block {1}) to {2} (block {3}) we have:".format(
    datetime.fromtimestamp(networks.active_provider.get_block(cutoff_block_start).timestamp, timezone.utc), 
    cutoff_block_start,
    datetime.fromtimestamp(networks.active_provider.get_block(cutoff_block_end).timestamp, timezone.utc),
    cutoff_block_end
))
print("There is a total of {0} users with deposits (eth or dai)".format(len(in_time)))
print("There is a total of {0} users with deposits (>{1} dai)".format(
    len(eligible[eligible["assetId"] == 1]),
    cutoff_dai / 1e18
))
print("There is a total of {0} users with deposits (>{1} eth)".format(
    len(eligible[eligible["assetId"] == 0]),
    cutoff_eth / 1e18
))
print('The winner is {0}, and have deposited {1} {2}'.format(
    winner[1], 
    winner[2] / 1e18, 
    "dai" if winner[0] == 1 else "eth"
))

For the timeframe 2022-07-22 12:00:16+00:00 (block 15192236) to 2022-07-29 12:00:03+00:00 (block 15237215) we have:
There is a total of 9927 users with deposits (eth or dai)
There is a total of 90 users with deposits (>2000.0 dai)
There is a total of 962 users with deposits (>1.0 eth)
The winner is 0xeb9Ac806d01487754Ff3cA8cFf89aaBb1068Fd83, and have deposited 1.10018 eth
